In [192]:
import Pkg; Pkg.activate(joinpath(@__DIR__,"..")); Pkg.instantiate()
using LinearAlgebra
using ForwardDiff
using RobotZoo
using RobotDynamics
using Ipopt
using MathOptInterface
using TrajOptPlots
const MOI = MathOptInterface
using Random
using Plots
include("quadruped.jl")
include("quadratic_cost.jl")
include("moi.jl")

  Activating environment at `C:\Users\pnadan\Documents\GitHub\Optimal-Control-Project\Project.toml`
Precompiling project...
  ✓ Wayland_protocols_jll
  ✓ OpenSSL_jll
  ✓ BenchmarkTools
  ✓ ChainRulesCore
  ✓ LERC_jll
  ✓ Expat_jll
  ✓ JSONSchema
  ✓ StructArrays
  ✓ ArrayInterface
  ✓ ChangesOfVariables
  ✓ Libtiff_jll
  ✓ WebIO
  ✓ Wayland_jll
  ✓ Fontconfig_jll
  ✓ RecipesPipeline
  ✓ Interpolations
  ✓ FiniteDiff
  ✓ PlotThemes
  ✓ LogExpFunctions
  ✓ xkbcommon_jll
  ✓ CSSUtil
  ✓ Cairo_jll
  ✓ JSExpr
  ✓ Qt5Base_jll
  ✓ StatsBase
  ✓ GeometryBasics
  ✓ HarfBuzz_jll
  ✓ SpecialFunctions
  ✓ Knockout
  ✓ MeshIO
  ✓ Blink
  ✓ DiffRules
  ✓ libass_jll
  ✓ FFMPEG_jll
  ✓ MechanismGeometries
  ✓ FFMPEG
  ✓ GR_jll
  ✓ InteractBase
  ✓ ForwardDiff
  ✓ GR
  ✓ SparseDiffTools
  ✓ MeshCat
  ✓ RobotDynamics
  ✓ MeshCatMechanisms
  ✓ RobotZoo
  ✓ MathOptInterface
  ✓ OSQP
  ✓ TrajectoryOptimization
  ✓ Ipopt
  ✓ Plots
  ✓ Altro
  ✓ TrajOptPlots
  52 dependencies successfully precompiled in 95 s

solve

## Dynamics Models

In [195]:
include("simple_quadruped.jl")
full_model = UnitreeA1()
n,m = state_dim(full_model), control_dim(full_model)
mvis = initialize_visualizer(full_model);
model = simplifyQuadruped(full_model)
n,m = state_dim(model), control_dim(model);

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://127.0.0.1:8710
└ @ MeshCat C:\Users\pnadan\.julia\packages\MeshCat\Ax8pH\src\visualizer.jl:73


## Reference Trajectory & Contact Sequence

In [196]:
include("ref_trajectory.jl")
include("gait.jl")

# Discretization
tf = 4.4
dt = 0.1
N = Int(ceil(tf/dt)) + 1
times = range(0,tf, length=N);

# Reference Trajectory
Xref,Uref = reference_trajectory(model, times)

# Contact Sequence
contact_sched = contact_sequence_trot(times)

LoadError: MethodError: no method matching reference_trajectory(::SimpleQuadruped, ::StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}})
[0mClosest candidates are:
[0m  reference_trajectory([91m::UnitreeA1[39m, ::Any; xinit, xterm, height) at C:\Users\pnadan\Documents\GitHub\Optimal-Control-Project\src\ref_trajectory.jl:1

## Define NLP for IPOPT

In [197]:
include("nlp.jl")
include("nlp_cost.jl")
include("nlp_constraint.jl")

LoadError: LoadError: UndefVarError: BlockViews not defined
in expression starting at C:\Users\pnadan\Documents\GitHub\Optimal-Control-Project\src\nlp.jl:1

## Solve NLP

## Controller

In [ ]:
A = [zeros(n,n) for k = 1:N-1]
B = [zeros(n,m) for k = 1:N-1]
for k = 1:N-1
    A[k], B[k] = discrete_jacobian(model, Xref[k], Uref[k], times[k], times[k+1]-times[k])
end
K, P = tvlqr(A,B,Q,R,Qf);


## Simulation